# Example 30 - Planar Dipping Layers

This example will show how to convert the geological map below using ``GemGIS`` to a `GemPy` model. This example is based on digitized data. The area is 1157 m wide (W-E extent) and 1361 m high (N-S extent). The vertical model extents varies between 0 m and 600 m. The model represents two planar stratigraphic units (blue and red) dipping towards the northeast above an unspecified basement (yellow). The map has been georeferenced with QGIS. The stratigraphic boundaries were digitized in QGIS. Strikes lines were digitized in QGIS as well and were used to calculate orientations for the `GemPy` model. These will be loaded into the model directly. The contour lines were also digitized and will be interpolated with `GemGIS` to create a topography for the model. 

Map Source: Unknown

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('../data/images/example30/cover_example30.png')
plt.figure(figsize=(10, 10))
imgplot = plt.imshow(img)
plt.axis('off')
plt.tight_layout()

## Licensing

Computational Geosciences and Reservoir Engineering, RWTH Aachen University, Authors: Alexander Juestel. For more information contact: alexander.juestel(at)rwth-aachen.de


This work is licensed under a Creative Commons Attribution 4.0 International License (http://creativecommons.org/licenses/by/4.0/)

## Import GemGIS

If you have installed ``GemGIS`` via pip or conda, you can import ``GemGIS`` like any other package. If you have downloaded the repository, append the path to the directory where the ``GemGIS`` repository is stored and then import ``GemGIS``. 

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import gemgis as gg

## Importing Libraries and loading Data

All remaining packages can be loaded in order to prepare the data and to construct the model. The example data is downloaded from an external server using `pooch`. It will be stored in a data folder in the same directory where this notebook is stored.  

In [ ]:
import geopandas as gpd
import rasterio

In [ ]:
file_path = '../data/example30_planar_dipping_layers/'

## Creating Digital Elevation Model from Contour Lines

The digital elevation model (DEM) will be created by interpolating contour lines digitized from the georeferenced map using the `SciPy` Radial Basis Function interpolation wrapped in `GemGIS`. The respective function used for that is `gg.vector.interpolate_raster()`. 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('../data/images/example30/dem_example30.png')
plt.figure(figsize=(10, 10))
imgplot = plt.imshow(img)
plt.axis('off')
plt.tight_layout()

In [ ]:
topo = gpd.read_file(file_path + 'topo30.shp')
topo.head()

### Interpolating the contour lines

In [ ]:
topo_raster = gg.vector.interpolate_raster(gdf=topo, value='Z', method='rbf', res=10)

### Plotting the raster

In [ ]:
import matplotlib.pyplot as plt

fix, ax = plt.subplots(1, figsize=(10, 10))
topo.plot(ax=ax, aspect='equal', column='Z', cmap='gist_earth')
im = plt.imshow(topo_raster, origin='lower', extent=[0, 1157, 0, 1361], cmap='gist_earth')
cbar = plt.colorbar(im)
cbar.set_label('Altitude [m]')
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.xlim(0, 1157)
plt.ylim(0, 1361)

### Saving the raster to disc

After the interpolation of the contour lines, the raster is saved to disc using `gg.raster.save_as_tiff()`. The function will not be executed as a raster is already provided with the example data. 

### Opening Raster

The previously computed and saved raster can now be opened using rasterio. 

In [ ]:
topo_raster = rasterio.open(file_path + 'raster30.tif')

## Interface Points of stratigraphic boundaries

The interface points will be extracted from LineStrings digitized from the georeferenced map using QGIS. It is important to provide a formation name for each layer boundary. The vertical position of the interface point will be extracted from the digital elevation model using the `GemGIS` function `gg.vector.extract_xyz()`. The resulting GeoDataFrame now contains single points including the information about the respective formation. 

In [ ]:
img = mpimg.imread('../data/images/example30/interfaces_example30.png')
plt.figure(figsize=(10, 10))
imgplot = plt.imshow(img)
plt.axis('off')
plt.tight_layout()

In [ ]:
interfaces = gpd.read_file(file_path + 'interfaces30.shp')
interfaces.head()

### Extracting Z coordinate from Digital Elevation Model

In [ ]:
interfaces_coords = gg.vector.extract_xyz(gdf=interfaces, dem=topo_raster)
interfaces_coords = interfaces_coords[interfaces_coords['formation'].isin(['A', 'B'])]
interfaces_coords

### Plotting the Interface Points

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 10))

interfaces.plot(ax=ax, column='formation', legend=True, aspect='equal')
interfaces_coords.plot(ax=ax, column='formation', legend=True, aspect='equal')
plt.grid()
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.xlim(0, 1157)
plt.ylim(0, 1361)

## Orientations from Strike Lines

Strike lines connect outcropping stratigraphic boundaries (interfaces) of the same altitude. In other words: the intersections between topographic contours and stratigraphic boundaries at the surface. The height difference and the horizontal difference between two digitized lines is used to calculate the dip and azimuth and hence an orientation that is necessary for `GemPy`. In order to calculate the orientations, each set of strikes lines/LineStrings for one formation must be given an id number next to the altitude of the strike line. The id field is already predefined in QGIS. The strike line with the lowest altitude gets the id number `1`, the strike line with the highest altitude the the number according to the number of digitized strike lines. It is currently recommended to use one set of strike lines for each structural element of one formation as illustrated. 


In [ ]:
img = mpimg.imread('../data/images/example30/orientations_example30.png')
plt.figure(figsize=(10, 10))
imgplot = plt.imshow(img)
plt.axis('off')
plt.tight_layout()

In [ ]:
strikes = gpd.read_file(file_path + 'strikes30.shp')
strikes

### Calculating Orientations for each formation

In [ ]:
orientations_a = gg.vector.calculate_orientations_from_strike_lines(gdf=strikes[strikes['formation'] == 'A'].sort_values(by='id', ascending=True).reset_index())
orientations_a

In [ ]:
orientations_b = gg.vector.calculate_orientations_from_strike_lines(gdf=strikes[strikes['formation'] == 'B'].sort_values(by='id', ascending=True).reset_index())
orientations_b

### Merging Orientations

In [ ]:
import pandas as pd
orientations = pd.concat([orientations_a, orientations_b])
orientations = orientations[orientations['formation'].isin(['A', 'B'])].reset_index()
orientations

### Plotting the Orientations

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 10))

interfaces.plot(ax=ax, column='formation', legend=True, aspect='equal')
interfaces_coords.plot(ax=ax, column='formation', legend=True, aspect='equal')
orientations.plot(ax=ax, color='red', aspect='equal')
plt.grid()
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.xlim(0, 1157)
plt.ylim(0, 1361)

## GemPy Model Construction

The structural geological model will be constructed using the `GemPy` package. 

In [ ]:
import gempy as gp

### Creating new Model

In [ ]:
geo_model = gp.create_model('Model30')
geo_model

### Initiate Data

In [ ]:
gp.init_data(geo_model, [0, 1157, 0, 1361, 0, 600], [50,50,50],
             surface_points_df=interfaces_coords,
             orientations_df=orientations,
             default_values=True)

### Model Surfaces

In [ ]:
geo_model.surfaces

### Mapping the Stack to Surfaces

In [ ]:
gp.map_stack_to_surfaces(geo_model,
                         {'Strata1': ('A'),
                          'Strata2': ('B')
                         },
                         remove_unused_series=True)
geo_model.add_surfaces('C')

# Adding additional orientations

In [ ]:
geo_model.add_orientations(X=1000, Y=1250, Z=500, surface='A', orientation = [217.5,18,1])
geo_model.add_orientations(X=100, Y=200, Z=300, surface='A', orientation = [217.5,7,1])
geo_model.add_orientations(X=400, Y=400, Z=200, surface='A', orientation = [217.5,14.5,1])

geo_model.add_orientations(X=850, Y=300, Z=200, surface='B', orientation = [217.5,26,1])
geo_model.add_orientations(X=1000, Y=100, Z=200, surface='B', orientation = [217.5,26,1])


### Showing the Number of Data Points

In [ ]:
gg.utils.show_number_of_data_points(geo_model=geo_model)

### Loading Digital Elevation Model

In [ ]:
geo_model.set_topography(source='gdal', filepath=file_path + 'raster30.tif')

### Plotting Input Data

In [ ]:
gp.plot_2d(geo_model, direction='z', show_lith=False, show_boundaries=False)
plt.grid()

In [ ]:
gp.plot_3d(geo_model, image=False, plotter_type='basic', notebook=True)

### Setting the Interpolator

In [ ]:
gp.set_interpolator(geo_model,
                    compile_theano=True,
                    theano_optimizer='fast_compile',
                    verbose=[],
                    update_kriging=False
                    )

### Computing Model

In [ ]:
sol = gp.compute_model(geo_model, compute_mesh=True)

### Plotting Cross Sections

In [ ]:
gp.plot_2d(geo_model, direction=['x', 'x', 'y', 'y'], cell_number=[25, 40, 25, 40], show_topography=True, show_data=False)

In [ ]:
gpv = gp.plot_3d(geo_model, image=False, show_topography=False,
                 plotter_type='basic', notebook=True, show_lith=True)